In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
import seaborn as sns

In [2]:
train_master = pd.read_csv("../data/master_train.csv")
test_master = pd.read_csv("../data/master_test.csv")

# master = pd.read_csv("../data/master.csv")

train_master = train_master.drop(['ENDOWMENT03', 'CPI_INDEX', 'HEPI_INDEX'], axis=1)
test_master = test_master.drop(['ENDOWMENT03', 'CPI_INDEX', 'HEPI_INDEX'], axis=1)

In [3]:
# TRAIN
nans_train = pd.isnull(train_master)
nans_copy_train = pd.isnull(train_master)

from collections import Counter

university_nans_train = Counter()
column_count_nans_train = Counter()

for index, row in zip(nans_copy_train.index, nans_copy_train.values):
    row_data = dict(zip(nans_copy_train.columns.values, row))
    for column, value in row_data.items():
        if value:
            university_nans_train[index] += 1
            column_count_nans_train[column] += 1
            
# A good number of our institutions have between 0 and 10 NaN values.
university_nan_dist_train = pd.DataFrame.from_dict(university_nans_train, orient='index').reset_index()
university_nan_dist_train = university_nan_dist_train.rename(columns={'index':'event', 0:'count'})
# plt.hist(university_nan_dist['count'])

# Feature to NaN DataFrame
column_nan_dist_train = pd.DataFrame.from_dict(column_count_nans_train, orient='index').reset_index()
column_nan_dist_train = column_nan_dist_train.rename(columns={'index':'event', 0:'count'})
column_nan_dist_train['count'] = column_nan_dist_train['count'].astype(int)

In [4]:
# TEST
nans_test = pd.isnull(test_master)
nans_copy_test = pd.isnull(test_master)

from collections import Counter

university_nans_test = Counter()
column_count_nans_test = Counter()

for index, row in zip(nans_copy_test.index, nans_copy_test.values):
    row_data = dict(zip(nans_copy_test.columns.values, row))
    for column, value in row_data.items():
        if value:
            university_nans_test[index] += 1
            column_count_nans_test[column] += 1
            
# A good number of our institutions have between 0 and 10 NaN values.
university_nan_dist_test = pd.DataFrame.from_dict(university_nans_test, orient='index').reset_index()
university_nan_dist_test = university_nan_dist_test.rename(columns={'index':'event', 0:'count'})
# plt.hist(university_nan_dist['count'])

# Feature to NaN DataFrame
column_nan_dist_test = pd.DataFrame.from_dict(column_count_nans_test, orient='index').reset_index()
column_nan_dist_test = column_nan_dist_test.rename(columns={'index':'event', 0:'count'})
column_nan_dist_test['count'] = column_nan_dist_test['count'].astype(int)

In [5]:
# pd.merge(column_nan_dist_test, column_nan_dist_train, on='event')

In [6]:
import statsmodels.stats.api as sms

# TRAIN
conf_interval_train = {feature: 0 for feature in column_nan_dist_train['event']}

for feature in conf_interval_train.keys():
    valid, invalid = train_master[train_master[feature].notnull()], train_master[train_master[feature].isnull()]
    valid, invalid = valid['DRate'], invalid['DRate']
    
    cm = sms.CompareMeans(sms.DescrStatsW(invalid), sms.DescrStatsW(valid))
    conf_interval_train[feature] = cm.tconfint_diff(usevar='unequal')
    
pprint(conf_interval_train)

{'ACADSUPP01': (4.2073726781828764, 5.2744692402709941),
 'CERTIFICATES_AWARDS_100FTE': (-5.4394269049486859, -4.3706866095833803),
 'FED_GRANT_PCT': (-9.118177489037528, -7.7975494835965193),
 'FT_FACULTY_PER_100FTE': (0.089825329286457589, 1.1157958877836527),
 'FT_FIRST_TIME_FIRST_YR_DEG_SEEK': (-8.9163401179046211, -7.5458261171946397),
 'GRANT01': (-9.0850808729801678, -7.5251171295098693),
 'GRANT02': (0.098737349705790067, 1.3752045471045764),
 'GRANT07': (-6.5473366752568554, -1.226904537572568),
 'INSTITUTIONAL_GRANT_AID_SHARE': (3.6583397314049324, 4.7954199918828389),
 'INST_GRANT_AVG_AMOUNT': (0.13274071321886916, 1.3035156280332953),
 'INST_GRANT_PCT': (-9.118177489037528, -7.7975494835965193),
 'LOAN_AVG_AMOUNT': (-9.0154342230598221, -7.5849931351842397),
 'LOAN_PCT': (-9.118177489037528, -7.7975494835965193),
 'NETTUITION01': (-8.8718579919394465, 5.4473438890714716),
 'NETTUITION_SHARE': (-9.3042697015481757, 2.6855786889799971),
 'NET_STUDENT_TUITION': (-8.87185799193

In [7]:
# TEST
conf_interval_test = {feature: 0 for feature in column_nan_dist_test['event']}

for feature in conf_interval_test.keys():
    valid, invalid = test_master[test_master[feature].notnull()], test_master[test_master[feature].isnull()]
    valid, invalid = valid['DRate'], invalid['DRate']
    
    cm = sms.CompareMeans(sms.DescrStatsW(invalid), sms.DescrStatsW(valid))
    conf_interval_test[feature] = cm.tconfint_diff(usevar='unequal')

cm = sms.CompareMeans(sms.DescrStatsW(invalid), sms.DescrStatsW(valid))
cm.tconfint_diff(usevar='unequal')

pprint(conf_interval_test)

{'ACADSUPP01': (2.9468323815252404, 3.918239701074814),
 'CERTIFICATES_AWARDS_100FTE': (-5.1912986044274039, -4.2147484832260815),
 'FED_GRANT_PCT': (-8.3915124144337074, -7.2701186331128351),
 'FT_FACULTY_PER_100FTE': (-2.7466361979861391, 0.84939674787693176),
 'FT_FIRST_TIME_FIRST_YR_DEG_SEEK': (-8.3070958907927928, -7.2022676154435974),
 'GRANT01': (-8.5122249717290615, -7.1860742839675815),
 'GRANT02': (-0.11059800344589832, 1.0221711080487603),
 'GRANT07': (-7.940656135553021, -4.6191981327804212),
 'INSTITUTIONAL_GRANT_AID_SHARE': (2.944134495205859, 3.9775000162928289),
 'INST_GRANT_AVG_AMOUNT': (-0.54360487596531248, 0.50453058122837069),
 'INST_GRANT_PCT': (-8.3915124144337074, -7.2701186331128351),
 'LOAN_AVG_AMOUNT': (-7.8615989673270192, -6.5843714881962985),
 'LOAN_PCT': (-8.3915124144337074, -7.2701186331128351),
 'NETTUITION01': (-7.3042550428807687, -0.13656375218785355),
 'NETTUITION_SHARE': (-6.7617804855448309, -0.3234938492409043),
 'NET_STUDENT_TUITION': (-7.30425

In [8]:
# TRAIN
imputable_train = [feature for feature, interval in conf_interval_train.items() 
                   if interval[0] <= 0 <= interval[1] or interval[1] >= 0 >= interval[0]]

non_imputable_train = [feature for feature in conf_interval_train.keys() if feature not in imputable_train]

In [9]:
# TEST
imputable_test = [feature for feature, interval in conf_interval_test.items() 
                   if interval[0] <= 0 <= interval[1] or interval[1] >= 0 >= interval[0]]

non_imputable_test = [feature for feature in conf_interval_test.keys() if feature not in imputable_test]

In [10]:
# TODO revisit if we decide to go with clustering
# TRAIN
train_master = train_master.drop(non_imputable_train, axis=1)

for feature in imputable_train:
    train_master[feature + '_missing'] = train_master[feature].isnull().astype(int)
    valid_values = train_master[feature].dropna()
    mean = np.mean(valid_values)
    train_master[feature].replace({np.nan: mean}, inplace=True)

In [11]:
# TEST
test_master = test_master.drop(non_imputable_test, axis=1)

for feature in imputable_test:
    test_master[feature + '_missing'] = test_master[feature].isnull().astype(int)
    valid_values = test_master[feature].dropna()
    mean = np.mean(valid_values)
    test_master[feature].replace({np.nan: mean}, inplace=True)

In [12]:
# TRAIN
categoricals_train = {}

for column in train_master.columns.values:
    values = set(train_master[column].values)
    if len(values) < 30 and len(values) != 2:
        categoricals_train[column] = values

categoricals_train

{'CENSUS_REGION': {1, 2, 3, 4, 5},
 'CONTROL_x': {1, 2, 3},
 'CONTROL_y': {1, 2, 3},
 'Ethnic Code': {1, 2, 3, 5},
 'HLOFFER': {1, 2, 3, 4, 5, 6, 7, 8, 9},
 'HOSPITAL': {-2, 1, 2},
 'ICLEVEL_x': {1, 2, 3},
 'ICLEVEL_y': {1, 2, 3},
 'INSTCAT': {-2, 1, 2, 3, 4, 5, 6},
 'INSTSIZE': {-2, -1, 1, 2, 3, 4, 5},
 'LOCALE': {11, 12, 13, 21, 22, 23, 31, 32, 33, 41, 42, 43},
 'Prog\nLength': {1, 2, 3, 4, 5, 6, 7, 8, 11, 12},
 'SECTOR_x': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
 'SECTOR_y': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
 'School\nType': {1, 2, 3}}

In [13]:
# TEST
categoricals_test = {}

for column in test_master.columns.values:
    values = set(test_master[column].values)
    if len(values) < 30 and len(values) != 2:
        categoricals_test[column] = values

categoricals_test

{'CENSUS_REGION': {1, 2, 3, 4, 5},
 'CONTROL_x': {1, 2, 3},
 'CONTROL_y': {1, 2, 3},
 'Ethnic Code': {1, 2, 3, 5},
 'HLOFFER': {1, 2, 3, 4, 5, 6, 7, 8, 9},
 'HOSPITAL': {-2, -1, 1, 2},
 'ICLEVEL_x': {1, 2, 3},
 'ICLEVEL_y': {1, 2, 3},
 'INSTCAT': {-2, 1, 2, 3, 4, 5, 6},
 'INSTSIZE': {-2, 1, 2, 3, 4, 5},
 'LOCALE': {11, 12, 13, 21, 22, 23, 31, 32, 33, 41, 42, 43},
 'Prog\nLength': {1, 2, 3, 4, 5, 6, 7, 8, 11, 12},
 'SECTOR_x': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
 'SECTOR_y': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
 'School\nType': {1, 2, 3}}

In [14]:
# TRAIN

# Dummify categoricals
for category in categoricals_train.keys():
    train_master = pd.concat([train_master, pd.get_dummies(train_master[category], prefix="{}_".format(category))], axis = 1)
    train_master = train_master.drop(category, axis = 1)

In [15]:
# TEST

# Dummify categoricals
for category in categoricals_test.keys():
    test_master = pd.concat([test_master, pd.get_dummies(test_master[category], prefix="{}_".format(category))], axis = 1)
    test_master = test_master.drop(category, axis = 1)

In [16]:
# TRAIN, TEST
train_master.to_csv("../data/train_master_imputed_dummified.csv")
test_master.to_csv("../data/test_master_imputed_dummified.csv")

In [21]:
sorted(test_master.columns.values)

['Average or\nGreater than 30',
 'CENSUS_REGION__1',
 'CENSUS_REGION__2',
 'CENSUS_REGION__3',
 'CENSUS_REGION__4',
 'CENSUS_REGION__5',
 'CONTROL_x__1',
 'CONTROL_x__2',
 'CONTROL_x__3',
 'CONTROL_y__1',
 'CONTROL_y__2',
 'CONTROL_y__3',
 'DRate',
 'Ethnic Code__1',
 'Ethnic Code__2',
 'Ethnic Code__3',
 'Ethnic Code__5',
 'FT_FACULTY_PER_100FTE',
 'FT_FACULTY_PER_100FTE_missing',
 'GRANT02',
 'GRANT02_missing',
 'GROFFER',
 'HAS_FTE',
 'HBCU_x',
 'HBCU_y',
 'HLOFFER__1',
 'HLOFFER__2',
 'HLOFFER__3',
 'HLOFFER__4',
 'HLOFFER__5',
 'HLOFFER__6',
 'HLOFFER__7',
 'HLOFFER__8',
 'HLOFFER__9',
 'HOSPITAL__-1',
 'HOSPITAL__-2',
 'HOSPITAL__1',
 'HOSPITAL__2',
 'HSI',
 'ICLEVEL_x__1',
 'ICLEVEL_x__2',
 'ICLEVEL_x__3',
 'ICLEVEL_y__1',
 'ICLEVEL_y__2',
 'ICLEVEL_y__3',
 'INSTCAT__-2',
 'INSTCAT__1',
 'INSTCAT__2',
 'INSTCAT__3',
 'INSTCAT__4',
 'INSTCAT__5',
 'INSTCAT__6',
 'INSTNAME',
 'INSTSIZE__-2',
 'INSTSIZE__1',
 'INSTSIZE__2',
 'INSTSIZE__3',
 'INSTSIZE__4',
 'INSTSIZE__5',
 'INST_GRA